https://www.kaggle.com/competitions/amex-default-prediction/data

In [1]:
import gc

In [2]:
import os
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
# from sklearn.model_selection import GridSearchCV, train_test_split

# from sklearn.pipeline import Pipeline
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# import xgboost as xgb
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import f1_score
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.impute import SimpleImputer
# # from imblearn.under_sampling import RandomUnderSampler

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
def calculate_correlation_with_target(features_df, target_series, variance_threshold=0.0):
    """
    Calculate the correlation of numeric columns in a features DataFrame with a target Series
    and perform Variance Threshold feature selection.
    
    Parameters:
    features_df (pd.DataFrame): The features DataFrame.
    target_series (pd.Series): The target Series.
    variance_threshold (float): Variance threshold for feature selection. Features with variance
        below this threshold will be removed. Default is 0.0 (no threshold).

    Returns:
    pd.Series: A Series containing the correlation coefficients sorted by absolute values.
    """
    # Select only numeric columns from the features DataFrame
    numeric_features = features_df.select_dtypes(include=['number'])
    
    # Calculate the correlation and sort the result by absolute values in descending order
    correlation_series = numeric_features.corrwith(target_series)
    absolute_correlation_series = correlation_series.abs()
    
    # Apply Variance Threshold to filter features
    if variance_threshold > 0.0:
        selector = VarianceThreshold(threshold=variance_threshold)
        numeric_features = selector.fit_transform(numeric_features)
        # Update correlation series to match the selected features
        correlation_series = pd.Series(selector.inverse_transform(correlation_series.values.reshape(1, -1))[0], index=numeric_features.columns)
    
    # Sort the DataFrame by absolute values
    correlation_series = correlation_series.sort_values(ascending=False)
    
    return correlation_series

In [5]:
# Define the directory path
data_dir = os.path.join('C:\\', 'Users', 'KonuTech', 'zoomcamp-capstone-01', 'data')
data_dir

'C:\\Users\\KonuTech\\zoomcamp-capstone-01\\data'

In [6]:
train_data_parquet_file = 'train_data.parquet'

In [7]:
# Load the training data
train_data = pd.read_parquet(os.path.join(data_dir, train_data_parquet_file))

In [8]:
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531451 entries, 0 to 5531450
Data columns (total 191 columns):
 #    Column       Dtype  
---   ------       -----  
 0    customer_ID  object 
 1    S_2          object 
 2    P_2          float64
 3    D_39         float64
 4    B_1          float64
 5    B_2          float64
 6    R_1          float64
 7    S_3          float64
 8    D_41         float64
 9    B_3          float64
 10   D_42         float64
 11   D_43         float64
 12   D_44         float64
 13   B_4          float64
 14   D_45         float64
 15   B_5          float64
 16   R_2          float64
 17   D_46         float64
 18   D_47         float64
 19   D_48         float64
 20   D_49         float64
 21   B_6          float64
 22   B_7          float64
 23   B_8          float64
 24   D_50         float64
 25   D_51         float64
 26   B_9          float64
 27   R_3          float64
 28   D_52         float64
 29   P_3          float64
 30   B_10         flo

### downsampling

In [9]:
# Assuming 'target' is your target column name
target_column = 'target'

# Assuming you have a DataFrame 'train_data' with the dataset

# Separate the dataset into two DataFrames based on the target value
zeros = train_data[train_data[target_column] == 0]
ones = train_data[train_data[target_column] == 1]

# Determine the size of the minority class (the one with fewer samples)
minority_class_size = min(len(zeros), len(ones))

# Sample an equal number of samples from each class
zeros_downsampled = zeros.sample(minority_class_size, random_state=42)
ones_downsampled = ones.sample(minority_class_size, random_state=42)

# Combine the downsampled DataFrames
downsampled_data = pd.concat([zeros_downsampled, ones_downsampled])

# Shuffle the downsampled dataset (optional but recommended)
downsampled_data = downsampled_data.sample(frac=1, random_state=42)

# Your downsampled dataset is in the 'downsampled_data' DataFrame

In [10]:
dfs = ["train_data", "zeros", "ones", "zeros_downsampled", "ones_downsampled"]

In [11]:
for el in dfs:
    if el in locals():
        del el

In [12]:
downsampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2755738 entries, 541332 to 2614482
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 3.9+ GB


In [13]:
downsampled_data.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,2.732522e+06,2.755738e+06,2.755738e+06,2.754627e+06,2.755738e+06,2.344286e+06,2.754627e+06,2.754627e+06,525770.000000,2.020065e+06,2.620816e+06,2.755738e+06,2.754627e+06,2.755738e+06,2.755738e+06,2.197122e+06,2.755738e+06,2.459355e+06,381430.000000,2.755625e+06,2.755738e+06,2.743884e+06,1.083252e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.738937e+06,2.566303e+06,2.755738e+06,8.993770e+05,2.755738e+06,2.755738e+06,2.755738e+06,2.754627e+06,2.755738e+06,2.344286e+06,2.755738e+06,2.755738e+06,2.672810e+06,1.107736e+06,2.723122e+06,2.755738e+06,2.755738e+06,1.249946e+06,2.755738e+06,2.690428e+06,2.755738e+06,2.523761e+06,2.753017e+06,2.755738e+06,2.447861e+06,2.755738e+06,2.754627e+06,1.457871e+06,2.755738e+06,2.754627e+06,284277.000000,2.754627e+06,2.624210e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.635546e+06,2.754627e+06,2.697527e+06,2.755738e+06,2.742263e+06,2.755738e+06,2.755738e+06,32596.000000,2.755738e+06,2.745929e+06,2.755738e+06,2.495960e+05,2.755738e+06,2.755737e+06,1.352836e+06,2.753017e+06,2.754627e+06,2.620816e+06,2.711416e+06,2.755738e+06,2.246600e+05,2.755738e+06,2.745929e+06,2.755738e+06,2.755738e+06,2.754627e+06,2.740658e+06,6.910000e+05,2.755738e+06,2.755709e+06,2.755738e+06,2.755738e+06,2.635546e+06,2.755737e+06,2.755738e+06,2.738937e+06,2.755738e+06,1.828170e+05,2.754627e+06,2.755738e+06,2.755738e+06,3518.0,2.755738e+06,2.755738e+06,5193.000000,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755699e+06,2.755738e+06,2.754627e+06,2.738937e+06,2.755738e+06,2.755738e+06,2.654897e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.755738e+06,2.747819e+06,2.755430e+06,2.747998e+06,2.745825e+06,2.755469e+06,2.731161e+06,2.697732e+06,2.697732e+06,1.344027e+06,3.785330e+05,2.697732e+06,2.755738e+06,2.755710e+06,4.376590e+05,2.684226e+06,2.754627e+06,1.784800e+04,2.754994e+06,22106.000000,22106.000000,23481.000000,2.754358e+06,2.755713e+06,2.132029e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.645693e+06,2.680601e+06,2.755738e+06,2.697732e+06,2.697732e+06,2.755440e+06,29326.000000,2.697732e+06,2.697732e+06,379288.000000,2.729592e+06,2.755738e+06,143460.000000,1.434600e+05,1.434600e+05,1.434600e+05,1.434600e+05,2.697732e+06,2.731177e+06,2.697732e+06,544492.000000,2.697732e+06,2.731114e+06,2.697732e+06,2755738.0
mean,5.697896e-01,1.813815e-01,1.719185e-01,5.090367e-01,1.266744e-01,2.619210e-01,8.880831e-02,1.911463e-01,0.223720,1.896464e-01,1.788027e-01,2.246005e-01,2.141091e-01,6.803190e-02,7.834897e-02,4.909192e-01,3.690383e-01,4.853733e-01,0.189523,1.173611e-01,2.436012e-01,5.440467e-01,1.523367e-01,1.101159e-01,2.684950e-01,1.557561e-01,1.573229e-01,5.771288e-01,1.823322e-01,1.007948e-01,9.856271e-02,1.555321e-01,2.197427e-01,9.733460e-01,5.161758e-02,2.629959e-01,8.027397e-02,3.005470e-01,3.744911e-01,1.863008e-01,9.098406e-02,5.943840e-02,2.833497e-01,8.260007e-02,1.149494e-01,3.842058e-01,3.594231e-01,5.27

In [14]:
downsampled_data.to_parquet(os.path.join(data_dir, "train_data_downsampled.parquet"))